In [1]:
import tensorflow as tf
import numpy as np
import sys
from PIL import Image
sys.path.append("models/models/research/")
sys.path.append("models/models/research/slim/")
from nets import resnet_v2
from slim.preprocessing import inception_preprocessing 
from datasets import imagenet
from tensorflow.contrib import slim

In [2]:
print(tf.__version__)

1.11.0


In [3]:
#预处理完后
x=tf.placeholder(tf.float32, shape=[1,224,224,3])

#adv为对抗样本 可以被训练和修改的量
#adv = tf.get_variable(name="adv", shape=[1,224,224,3], dtype=tf.float32, initializer=tf.zeros_initializer)

with slim.arg_scope(resnet_v2.resnet_arg_scope()):
    logits, _ = resnet_v2.resnet_v2_50(x, num_classes=1001, is_training=False)
probabilities = tf.nn.softmax(logits)

saver=tf.train.Saver()

sess=tf.Session()

saver.restore(sess,'models/resnet_v2_50/resnet_v2_50.ckpt')


Instructions for updating:
keep_dims is deprecated, use keepdims instead
INFO:tensorflow:Restoring parameters from models/resnet_v2_50/resnet_v2_50.ckpt


In [4]:
path = "../picture/pig.jpg"
img=Image.open(path)

img=img.resize((224,224))
img=np.array(img).copy().astype(np.float32)

#tf中imagenet数据标准化就是归一化
img /= 255.0
img=np.expand_dims(img, 0)


In [5]:
predictions = sess.run([probabilities],{x:img})
predictions=np.squeeze(predictions)
#argsort是从小到大排序 所以要取排名倒数的
top_k = predictions.argsort()[-5:][::-1]
#imagenet关系映射表
names = imagenet.create_readable_names_for_imagenet_labels()

for node_id in top_k:
      human_string = names[node_id]
      score = predictions[node_id]
      print('%s (score = %.5f)' % (human_string, score))

hog, pig, grunter, squealer, Sus scrofa (score = 0.98853)
piggy bank, penny bank (score = 0.01128)
wild boar, boar, Sus scrofa (score = 0.00015)
French bulldog (score = 0.00001)
Chihuahua (score = 0.00000)
